In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8

  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
import io
import wave
from base64 import b64decode
from IPython.display import Audio, display, HTML
from google.colab import output

In [ ]:
def save_audio_as_wav(audio, filename):
    with wave.open(filename, 'wb') as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(44100)
        wav_file.writeframes(audio)

In [ ]:
import os

def save_recorded_audio(file_name):
    audio, sr = get_audio()

    file_path = os.path.join('/content/drive/Shareddrives/BME_AI/speech2text/', file_name)

    save_audio_as_wav(audio, file_path)
    print(f'녹음된 오디오를 WAV 파일로 저장했습니다: {file_path}')

    return file_path

In [ ]:
import numpy as np

file_name = 'recording.wav'

file_path = save_recorded_audio(file_name)

with wave.open(file_path, 'rb') as wav_file:
    frames = wav_file.readframes(wav_file.getnframes())
    audio_data = np.frombuffer(frames, dtype=np.int16)

녹음된 오디오를 WAV 파일로 저장했습니다: /content/drive/Shareddrives/BME_AI/speech2text/recording.wav


In [ ]:
!pip install SpeechRecognition

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()
kr_audio = sr.AudioFile('/content/drive/Shareddrives/BME_AI/speech2text/recording.wav')

with kr_audio as source:
    audio = r.record(source)

#sys.stdout = open('news_out.txt', 'w')
print(r.recognize_google(audio, language='ko-KR'))

#sys.stdout.close()

더 아픈 곳은 없어요


In [ ]:
!pip install PyKakao --upgrade

In [ ]:
from PyKakao import KoGPT
api = KoGPT(service_key = "88859c17b103e0e8687565c00348f065")

In [ ]:
basic = '''환자 정보: 질병 루게릭병, 성별 남, 나이 30대, 상태 감기
환자 정보를 바탕으로 질문에 답하세요'''
question = '''Q:'''+ r.recognize_google(audio, language='ko-KR')
answer = '''A:'''

prompt = basic + question + answer

response = api.generate(prompt, max_tokens=64, temperature=0.5, top_p=0.85, n = 6)

In [ ]:
results = response.get('generations')
results

[{'text': ' 목이 뻣뻣하고 숨쉬기가 힘들어 병원을 찾았습니다. 의사 선생님이 목 부위가 많이 아프고 근육도 굳었다고 하셨는데, 이 정도면 루게릭병인가요? 루게릭병은 어떤 병인지 궁금합니다. 또 증상 완화법을 알려주세요. 그리고 최근',
  'tokens': 64},
 {'text': '조금씩 움직이는 것이 불편해요. 팔을 머리 위로 올리지 못하고 발뒤꿈치를 들고 있습니다. 손도 잘 올라가지 않아요. 몸의 근육들이 점점 굳어가면서 마비가 오기 시작했어요. 아침저녁마다 재활운동과 치료받으러 가는데 이제는 갈 때',
  'tokens': 64},
 {'text': '몸이 많이 아프고 힘들어서 병원에서 처방해준 약을 먹고 있습니다. 그런데 다른 사람들보다 몸이 너무 많이 아파요. 아침에 일어나면 머리가 깨질 듯이 아프고, 손과 발에 힘이 들어가지 않아요. 그리고 눈도 잘 보이지 않고요. 그래서 밥',
  'tokens': 64},
 {'text': '몸이 많이 안 좋아요. 몸이 아프면 마음도 같이 아파요. Q:언제부터 그랬나요? A:2년 전쯤인 것 같은데... Q:어떤 증상인가요? A:처음엔 조금만 걸어도 다리가 아팠는데 지금은 걷는 게 힘들고 앉아 있어도 숨',
  'tokens': 64},
 {'text': ' 현재 증상이 있나요?네. 기침과 가래가 나오고 콧물도 조금 나요. 하지만 다른 사람들보다 심하진 않아요. 목이 많이 붓는 편인데 가끔씩 숨쉬기가 힘들 때가 있지만 일상생활엔 지장이 없습니다. Q: 식사할 수 있으세요? A:',
  'tokens': 64},
 {'text': '처음엔 손발이 많이 저리고 힘도 잘 안 들어가서 힘들었는데 지금은 괜찮아요. Q:가족 중에도 비슷한 병을 앓는 사람 있나요? A:없습니다. 부모님 모두 건강하시고요. 제가 제일 먼저 죽는다는 생각만 하면 너무 슬퍼요. 하지만 저',
  'tokens': 64}]

In [ ]:
left = []
right = []
for i in range(len(results)):
    if i < len(results)/2:
        left.append(results[i].get('text').split('Q')[0].rstrip().replace("\n",""))

    else:
        right.append(results[i].get('text').split('Q')[0].rstrip().replace("\n",""))
left, right

([' 목이 뻣뻣하고 숨쉬기가 힘들어 병원을 찾았습니다. 의사 선생님이 목 부위가 많이 아프고 근육도 굳었다고 하셨는데, 이 정도면 루게릭병인가요? 루게릭병은 어떤 병인지 궁금합니다. 또 증상 완화법을 알려주세요. 그리고 최근',
  '조금씩 움직이는 것이 불편해요. 팔을 머리 위로 올리지 못하고 발뒤꿈치를 들고 있습니다. 손도 잘 올라가지 않아요. 몸의 근육들이 점점 굳어가면서 마비가 오기 시작했어요. 아침저녁마다 재활운동과 치료받으러 가는데 이제는 갈 때',
  '몸이 많이 아프고 힘들어서 병원에서 처방해준 약을 먹고 있습니다. 그런데 다른 사람들보다 몸이 너무 많이 아파요. 아침에 일어나면 머리가 깨질 듯이 아프고, 손과 발에 힘이 들어가지 않아요. 그리고 눈도 잘 보이지 않고요. 그래서 밥'],
 ['몸이 많이 안 좋아요. 몸이 아프면 마음도 같이 아파요.',
  ' 현재 증상이 있나요?네. 기침과 가래가 나오고 콧물도 조금 나요. 하지만 다른 사람들보다 심하진 않아요. 목이 많이 붓는 편인데 가끔씩 숨쉬기가 힘들 때가 있지만 일상생활엔 지장이 없습니다.',
  '처음엔 손발이 많이 저리고 힘도 잘 안 들어가서 힘들었는데 지금은 괜찮아요.'])